In [ ]:

%%configure -f
{
  "conf": {
    "spark.jars": "abfss://f7e2db24-d041-47bb-ae02-0c67982c6c4f@onelake.dfs.fabric.microsoft.com/8fcfa082-2992-46d9-9794-dea041d8c11a/Files/Drivers/neo4j-spark-connector-5.3.8-s_2.12.jar"
  }
}



In [ ]:
%%spark
//JSON File in lakehouse with Neo4j credentials
val absfss_JSON_Base_Path = "abfss://f7e2db24-d041-47bb-ae02-0c67982c6c4f@onelake.dfs.fabric.microsoft.com/8fcfa082-2992-46d9-9794-dea041d8c11a/Files/Drivers/neo4j-conn.json"

// Import required libraries
import org.apache.spark.sql.{DataFrame, SaveMode, SparkSession}
import com.fasterxml.jackson.databind.ObjectMapper
import com.fasterxml.jackson.module.scala.DefaultScalaModule
import com.fasterxml.jackson.module.scala.experimental.ScalaObjectMapper

// Load JSON file for Neo4j credentials
val jsonString = spark.read.text(absfss_JSON_Base_Path).as[String].collect().mkString("\n")

// Parse JSON string
val mapper = new ObjectMapper() with ScalaObjectMapper
mapper.registerModule(DefaultScalaModule)
val data = mapper.readValue[Map[String, Any]](jsonString)

// Extract Neo4j connection details
val neo4jUrl = data("neo4j-url").asInstanceOf[String]
val neo4jUsername = data("neo4j-username").asInstanceOf[String]
val neo4jPassword = data("neo4j-password").asInstanceOf[String]

val spark = SparkSession.builder 
    .config("spark.neo4j.bolt.url", neo4jUrl) 
    .config("spark.neo4j.bolt.user", neo4jUsername)
    .config("spark.neo4j.bolt.password", neo4jPassword) 
    .getOrCreate()


In [ ]:
%%spark

val df = spark.read.format("org.neo4j.spark.DataSource")  
    .option("query", "match(o:Order)-[oc:ORDER_CONTAINS]->(p:Product) WHERE p.productID = \"41\" RETURN o.customerID, o.employeeID, o.freight, o.orderDate, o.orderID, o.requiredDate, o.shippedDate, p.productID") 
    .option("url", neo4jUrl)
    .option("authentication.basic.username", neo4jUsername)
    .option("authentication.basic.password", neo4jPassword)
    .load()

df.show()

In [ ]:
%%spark
//Write to Files/
df.write.mode("overwrite").format("csv").option("header", "true").save("Files/ExportFromNeo4j/test.csv")


In [ ]:
   %%spark
   //Write to Tables/
   df.write.format("delta").mode("overwrite").saveAsTable("Lakehouse1.Analysis")